In [1]:
#Importing required dependencies

import tweepy
import pymongo
import json
import time

print("Dependencies imported")

Dependencies imported


In [2]:
#TO DO: Add to config file later

consumer_key = "mQXkO4dzpD3OPD1nOIT3fuD0j"
consumer_secret = "obt5r2qT1o878i5Eja7GYropJfCeUX76IXcjXRuHoJjIZnaard"
users_database_name = "users_test"
user_followers_database = "user_followers_test"
tweets_database = "user_tweets_test"

seed_usernames = ["P9Ashwini", 
                  "drashwathcn", 
                  "Tej_AnanthKumar", 
                  "vijeta_at", 
                  "nimmasuresh", 
                  "ShashikalaJolle"]

In [3]:
#Storing in MongoDB database

client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["mydatabase_test"]
user_col = mydb[users_database_name]
user_follower_col = mydb[user_followers_database]
tweets_col = mydb[tweets_database]

In [4]:
#Checking for collections in DB

collist = mydb.list_collection_names()
print(collist)
if users_database_name in collist:
    print("Users collection exists")

else:
    print("ERR: Users collection doesn't exist yet")

[]
ERR: Users collection doesn't exist yet


In [5]:
#OAuth2 Authentication

def get_authenticated_twitter_api():
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

In [6]:
#Save the users info and recent tweets in local DB

def save_user(user_dict):
    try:
        user_info = {"_id":user_dict["id_str"],
                     "name":user_dict["name"], 
                     "screen_name":user_dict["screen_name"], 
                     "followers":user_dict["followers_count"], 
                     "friends":user_dict["friends_count"]}

        x = user_col.insert_one(user_info)

        #Fetching tweets of the user
        tweets = api.user_timeline(seed_user)
        for tweet_obj in tweets:
            tweet = tweet_obj._json
            tweet_info = {"_id":tweet["id_str"],
                          "user_id":user_dict["id_str"],
                          "follower_count":user_dict["followers_count"],
                          "retweet_count":tweet["retweet_count"]}

            y = tweets_col.insert_one(tweet_info)
            
    except:
        print("User already saved")

In [7]:
#Performing BFS on twitter users

def scrape_users(user_list):
    count = 0
    try:
        while len(user_list)!=0:
            if count%500==0:
                if(count!=0):
                    print("Pausing...")
                    time.sleep(300)
                api = get_authenticated_twitter_api()

            while(1):
                user_id = user_list.pop(0)
                
                #Checking if user node not already visited
                query = {"_id": user_id }
                query_res = user_col.find(query)
                
                qcount = 0
                for query in query_res:
                    qcount = 1
                
                if (qcount)==0:
                    break
                
            followers = []
            for follower in tweepy.Cursor(api.followers, id=user_id).items():
                if "bangalore" in follower._json["location"].lower() or "bengaluru" in follower._json["location"].lower():
                    user_list.append(follower._json["id_str"])
                    followers.append(follower._json["id_str"])
                    save_user(follower._json)
                    count = count+1

            #Saving followers of a user        
            follower_info = {"_id":user_id, 
                             "followers": followers }

            try:
                user_follower_col.insert_one(follower_info)

            except:
                print("User's followers already stored")

            if count>5000:
                break
                
    except Exception as ex:
        print(ex)

In [8]:
#Fetching user ids of popular seed users

def initialise(seed_usernames):
    seed_users = []
    api = get_authenticated_twitter_api()
    for seed in seed_usernames:
        res = api.get_user(seed)
        #print(res)
        seed_users.append(res._json["id_str"])
        
    print(seed_users)
    return seed_users

In [ ]:
#Fetching seed ids

seed_users = initialise(seed_usernames)

In [ ]:
#Drop the existing collections

user_col.drop()
user_follower_col.drop()
tweets_col.drop()

In [ ]:
seed_users = ['1213502011', '2885971046', '1067115424339435520', '2307766760', '43705720', '704994405065760768']

scrape_users(seed_users)

In [183]:
for x in user_col.find():
    print(x) 